In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
import os
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/MLProject_KenyaFinancial/official.csv" #replace with your data path
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MLProject_KenyaFinancial


'/content/drive/MyDrive/MLProject_KenyaFinancial'

In [4]:
data = pd.read_csv(DATA_PATH)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18,26,55,189,190,192,194,196,197,201) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data.head()

,Unnamed: 0,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,0,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#data.columns

Index(['Unnamed: 0', 'hh_ids', 'unique_hhs', 'first_trx_date_hh',
       'last_trx_date_hh', 'tot_hh_daysofobs', 'tot_hh_monthsofobs',
       'interview_designation', 'int_date', 'int_month',
       ...
       'dem_i_age_workage', 'dem_i_relhead', 'dem_i_hhead', 'dem_i_marstat',
       'dem_i_tribe', 'dem_i_children_tot', 'dem_i_children_inhh',
       'edu_i_enrolled', 'edu_i_attain', 'edu_i_attain_yrs'],
      dtype='object', length=203)

In [19]:
#pd.get_option("display.max_columns")

In [16]:
#pd.set_option("display.max_columns", None)

In [6]:
dataX = data.drop(['trx_mode_desc', 'trx_mode_code'], axis = 1)

In [7]:
X = dataX.loc[:,:]

In [9]:
X.columns

Index(['Unnamed: 0', 'hh_ids', 'unique_hhs', 'first_trx_date_hh',
       'last_trx_date_hh', 'tot_hh_daysofobs', 'tot_hh_monthsofobs',
       'interview_designation', 'int_date', 'int_month',
       ...
       'dem_i_age_workage', 'dem_i_relhead', 'dem_i_hhead', 'dem_i_marstat',
       'dem_i_tribe', 'dem_i_children_tot', 'dem_i_children_inhh',
       'edu_i_enrolled', 'edu_i_attain', 'edu_i_attain_yrs'],
      dtype='object', length=201)

In [10]:
Y = data.trx_mode_desc

In [8]:
for column in X.columns:
  if X[column].dtype == "object":
    X[column] = X[column].astype("|S")

In [59]:
#X = X.infer_objects()

In [12]:
#X.dtypes

Unnamed: 0               int64
hh_ids                     |S7
unique_hhs             float64
first_trx_date_hh          |S9
last_trx_date_hh           |S9
                        ...   
dem_i_children_tot     float64
dem_i_children_inhh    float64
edu_i_enrolled         float64
edu_i_attain              |S33
edu_i_attain_yrs       float64
Length: 201, dtype: object

In [9]:
dropCols = []
for column in X.columns:
  if X[column].nunique() > 200:
    dropCols.append(column)
X = X.drop(dropCols, axis=1)

In [10]:
#dropCols

['Unnamed: 0',
 'hh_ids',
 'int_date',
 'account_ids',
 'm_ids_owner',
 'first_trx_date_acc',
 'last_trx_date_acc',
 'tot_acc_daysofobs',
 'trx_id',
 'm_ids_trx',
 'trx_date',
 'trx_stdtime_days_hh',
 'trx_stdtime_days_acc',
 'trx_distance_km',
 'trx_outlet',
 'trx_value_kes',
 'trx_value_usd',
 'trx_inkind_units',
 'trx_inkind_value_usd',
 'trx_inkind_value_kes',
 'trx_stdtime_mnths_hh_nr',
 'con_pur_hh_byti_x',
 'con_pur_hh_byti_CLN',
 'con_pur_hh_byti_COMM',
 'con_pur_hh_byti_EDU',
 'con_pur_hh_byti_ENRGY',
 'con_pur_hh_byti_FOOD',
 'con_pur_hh_byti_H20',
 'con_pur_hh_byti_MED',
 'con_pur_hh_byti_OTH',
 'con_pur_hh_byti_PERS',
 'con_pur_hh_byti_TRNSP',
 'inc_hh_byti_x',
 'inc_hh_byti_AGRI',
 'inc_hh_byti_CITMPC',
 'inc_hh_byti_REI',
 'inc_hh_byti_RIOI',
 'inc_hh_byti_RR',
 'inc_hh_byti_SEI',
 'inc_hh_byti_y',
 'con_pur_hh_byti_y',
 'con_prod_hh_byti_x',
 'con_hh_byti_x',
 'inc_hh_byti_pae_oecd_x',
 'con_hh_byti_pae_oecd_x',
 'inc_hh_byti_pae_kihbs_x',
 'con_hh_byti_pae_kihbs_x',
 'i

In [25]:
#Skip and load X with pickle in later cell

from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder(handle_unknown='ignore', drop='if_binary')

X = ohc.fit_transform(X)

In [26]:
#X.shape

(736518, 3679)

In [27]:
with open("Xpkl", 'wb') as ofile:
  pickle.dump(X, ofile)


In [11]:
####################################### LOADING X
with open("Xpkl", 'rb') as ifile:
  X = pickle.load(ifile)

In [ ]:
############################### Program is crashing here itself due to lack of ram
xdf = pd.DataFrame(X.toarray())
#xdf.to_csv("X.csv")

In [21]:
# Encoding for Y needs to be done here

In [18]:
############################## Adding numeric columns back to X
for col in dropCols:
  xdf[col] = dataX[col]

In [3]:
#xdf.shape
xdf.head#

NameError: ignored

In [38]:
#type(X)

scipy.sparse.csr.csr_matrix

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [44]:
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train, y_train)

ValueError: ignored